In [1]:
import torch as t
import torchvision
import numpy as np
import matplotlib.pyplot as plt
import os
import Preprocesado as pre
import torch.nn as nn

clases_equialencia=[
    "A","B","C","D","E","F","G",
    "H","I","J","K","L","M","N",
    "Ñ","O","P","Q","R","S","T",
    "U","V","W","X","Y","Z",

    # "a","b","c","d","e","f","g",
    # "h","i","j","k","l","m","n",
    # "ñ","o","p","q","r","s","t",
    # "u","v","w","x","y","z"
]

clases=range(len(clases_equialencia))

def EquivalenciaEnteroAClase(entero):
    return clases_equialencia[entero]

def EquivalenciaClaseAEntero(clase):
    return clases[clases_equialencia.index(clase)]


In [2]:
directorioDataset="./Dataset"
dimEntradaIA=32
épocas=1
coeficienteAprendizaje=0.001

#no cambiar
tamañoLote=1

if directorioDataset=="":
    print("No se ha especificado una fuente")
    exit() #return 0

#verificar si la fuente es un directorio (si es un archivo, se cancela el proceso)
if not os.path.isdir(directorioDataset):
    print("La fuente no es un directorio")
    exit() #return 0

#creamos una lista con los archivos del directorio
archivos=os.listdir(directorioDataset)

if len(archivos)==0:
    print("El directorio está vacío")
    exit() #return 0

#para cada archivo, se hace el preprocesado y se almacena el objeto Resultado en una lista
dataset=[]
#el siguiente for tiene (0,len(archivos)) porque el último valor no se toma en cuenta (es decir, len(archivos)-1
for i in range(0,len(archivos)):
    res=pre.Preprocesar(fuente= directorioDataset+"/"+archivos[i],destino=directorioDataset+"/serilizados/",dimensionEntradaIA=dimEntradaIA)
    if not res.preprocesadoExitoso:
        print("El archivo "+archivos[i]+" no pudo ser preprocesado correctamente")
    else:
        for i in range(len(res.serie)):
            dataset.append((EquivalenciaClaseAEntero(res.serie[i][0]),t.tensor(res.serie[i][1],dtype=t.float32)))

print("Dataset cargado")


Preprocesando  ./Dataset/modeloEntrenado.pth ...
Validando imagen...
No se pudo leer la imagen
Validación fallida
El archivo modeloEntrenado.pth no pudo ser preprocesado correctamente
Preprocesando  ./Dataset/PlantillaMayus.png ...
Validando imagen...
Imagen validada
Reorientando imagen...
Reorientación exitosa
Segmentando imagen...
Analizando QR...
QR analizado
Segmentación exitosa
Preprocesamiento de  ./Dataset/PlantillaMayus.png  exitoso
Dataset cargado


In [3]:
import Lenet5 as ln5
# modelo=ln5.GenericLenet()
modelo=ln5.Lenet5()

# cargadorEntrenamiento=t.utils.data.DataLoader(dataset,batch_size=tamañoLote*len(clases),shuffle=False)
cargadorEntrenamiento=t.utils.data.DataLoader(dataset,batch_size=tamañoLote,shuffle=False)

funcionError=nn.CrossEntropyLoss()
functionOptimizador=t.optim.SGD(modelo.parameters(),lr=coeficienteAprendizaje)

#se entrena la red neuronal
'''
pasos: la cantidad de cuadros a analizar en cada iteración (época)
época: la cantidad de veces que se analizará todo el dataset
tamaño del lote: la cantidad de cuadros que se analizarán en cada iteración al mismo tiempo (debe ser 1 porque se especificó que el canal de entrada es 1) g
'''
superiteración=-1
pasos=len(cargadorEntrenamiento)
for época in range(épocas):
    print(f'Época {época+1}/{épocas}-------------------------------------------')
    for i,(etiquetas,imagenes) in enumerate(cargadorEntrenamiento):
        superiteración+=1
        print(f"Iteración {i}")
        print(f"Súper iteración {superiteración}")
        #se hace el forward
        salidas=modelo(imagenes)
        #se calcula el error
        error=funcionError(salidas,etiquetas)
        #se hace el backward
        functionOptimizador.zero_grad()
        error.backward()
        #se actualizan los pesos
        functionOptimizador.step()
                    
        if (i+1)%pasos==0:
            print(f'Paso {i+1}/{pasos}, Error={error.item():.4f}')

print("Entrenamiento finalizado")

with t.no_grad():
    n_correctas=0
    n_muestras=0
    n_clases_correctas=[0 for i in range(len(clases))]
    n_clases_muestras=[0 for i in range(len(clases))]

    for etiquetas,imagenes in cargadorEntrenamiento:
        salidas=modelo(imagenes)
        _,predicciones=t.max(salidas,1)
        n_muestras+=etiquetas.size(0)
        n_correctas+=(predicciones==etiquetas).sum().item()

        for i in range(len(etiquetas)):
            etiqueta=etiquetas[i]
            prediccion=predicciones[i]
            if etiqueta==prediccion:
                n_clases_correctas[etiqueta]+=1
            n_clases_muestras[etiqueta]+=1

print(f'Exactitud del entrenamiento: {100*n_correctas/n_muestras}%')

for i in range(len(clases)):
    exactitud_clase=100*n_clases_correctas[i]/n_clases_muestras[i]
    print(f'Exactitud de la clase {clases[i]}: {exactitud_clase}%')

t.save(modelo.state_dict(),directorioDataset+"/modeloEntrenado.pth")
print("Modelo guardado")

Época 1/1-------------------------------------------
Iteración 0
Súper iteración 0
1x32x32
C1
S2
C3
S4
C5
F6
F7
Iteración 1
Súper iteración 1
1x32x32
C1
S2
C3
S4
C5
F6
F7
Iteración 2
Súper iteración 2
1x32x32
C1
S2
C3
S4
C5
F6
F7
Iteración 3
Súper iteración 3
1x32x32
C1
S2
C3
S4
C5
F6
F7
Iteración 4
Súper iteración 4
1x32x32
C1
S2
C3
S4
C5
F6
F7
Iteración 5
Súper iteración 5
1x32x32
C1
S2
C3
S4
C5
F6
F7
Iteración 6
Súper iteración 6
1x32x32
C1
S2
C3
S4
C5
F6
F7
Iteración 7
Súper iteración 7
1x32x32
C1
S2
C3
S4
C5
F6
F7
Iteración 8
Súper iteración 8
1x32x32
C1
S2
C3
S4
C5
F6
F7
Iteración 9
Súper iteración 9
1x32x32
C1
S2
C3
S4
C5
F6
F7
Iteración 10
Súper iteración 10
1x32x32
C1
S2
C3
S4
C5
F6
F7
Iteración 11
Súper iteración 11
1x32x32
C1
S2
C3
S4
C5
F6
F7
Iteración 12
Súper iteración 12
1x32x32
C1
S2
C3
S4
C5
F6
F7
Iteración 13
Súper iteración 13
1x32x32
C1
S2
C3
S4
C5
F6
F7
Iteración 14
Súper iteración 14
1x32x32
C1
S2
C3
S4
C5
F6
F7
Iteración 15
Súper iteración 15
1x32x32
C1
S2
C3
S4


d:\Work\Dev\Repo\lenet5\Caligraphy_with_Lenet-5\Lenet5.py:259: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  x = self.activacionSalida(x)
